In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from decimal import *

In [6]:
pred = pd.read_csv("checkpoint2_submissions/prophet_predictions.csv")

pred.head()

,id,10,20,30,40,50,60,70,80,90
0,2020-05-10-1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-05-11-1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-05-12-1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-05-13-1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-05-14-1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
pred.loc[pred["id"] == "2020-04-07-36061"].to_numpy()[0][1:]

array([0.0, 102.65268191818024, 181.05126707807815, 249.49447634465577,
       311.71557567790813, 373.9366750111605, 442.3798842777381,
       520.778469437636, 630.2876042641602], dtype=object)

In [14]:
key = pd.read_csv("data/us/processing_data/fips_key.csv", encoding='latin-1')
all_fips = key["FIPS"].tolist()

print('2020-05-10'+'-'+str(all_fips[14]))

2020-05-10-1029


In [22]:
data = {}
for fips in all_fips:
    data[fips] = {}
    
    dates = ['2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17','2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25','2020-05-26']
    for i, date in enumerate(dates):
        id_str = date + '-' + str(fips)
        data[fips][date] = pred.loc[pred["id"] == id_str].to_numpy()[0][1:]

NameError: name 'get_id_list' is not defined

In [23]:
print(data[6085]['2020-05-10'])

[0.0 0.0 0.0 1.3380747377518512 1.724823943175019 2.1115731485981866
 2.5369972745636717 3.0243012733968633 3.704979874941638]


In [25]:
# We import the sample_submission.csv file as a way of determining
# the order of the rows in out output file
sample_submission = pd.read_csv("sample_submission.csv")
daily_deaths = pd.read_csv("data/us/covid/nyt_us_counties_daily.csv")
cumulative_deaths = pd.read_csv("data/us/covid/deaths.csv")

# Relevant dates
today = cumulative_deaths.columns[-1]
yesterday = cumulative_deaths.columns[-2]
one_week_ago = cumulative_deaths.columns[-8]
two_weeks_ago = cumulative_deaths.columns[-15]
beginning = cumulative_deaths.columns[4]

print("Today: " + today)
print("One week ago: " + one_week_ago)

# Assume date is in format mm/dd/yy, convert to yyyy-mm-dd
def convert_date_to_yyyy_mm_dd(date):
    parts = date.split('/')
    
    # Ensure leading zeros if necessary
    if len(parts[0]) == 1:
        parts[0] = "0" + parts[0]
    
    if len(parts[1]) == 1:
        parts[1] = "0" + parts[1]
        
    return "2020" + "-" + parts[0] + "-" + parts[1]

# Assume date is in format yyyy-mm-dd, convert to mm/dd/yy
def convert_date_to_mm_dd_yy(date):
    parts = date.split('-')
    
    # Remove leading zeros if necessary
    if parts[1][0] == "0":
        parts[1] = parts[1][1:]
    
    if parts[2][0] == "0":
        parts[2] = parts[2][1:]
        
    return parts[1] + "/" + parts[2] + "/" + "20"

# Get the name of a county from a given FIPS code
def get_name_from_fips(FIPS):
    return key.loc[key["FIPS"] == FIPS]["COUNTY"].values[0]

# Get the date of the first death of a given county. If the
# county has no deaths, return "N/A"
def get_date_of_first_death(FIPS):
    county = cumulative_deaths.loc[cumulative_deaths["countyFIPS"] == FIPS]
    deaths_dates = county.drop(columns=['countyFIPS', 'County Name', 'State', 'stateFIPS'])
    lst = deaths_dates.values[0]

    for i in range(len(lst)):
        if lst[i] != 0:
            return deaths_dates.columns[i]

    return "N/A"

# Returns true if there exists deaths data for the county,
# and false otherwise. We need this because some FIPS are
# not included in the ny times data at all.
def deaths_data_exists(FIPS):
    return len(daily_deaths.loc[daily_deaths["fips"] == FIPS].values) != 0

# Generate the quantiles for a given value and standard error
# according to a normal distribution.
def generate_quantiles(value, err):
    if err == 0:
        return [value] * 9
    
    quantiles = []
    for quantile in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        quantiles.append(norm.ppf(quantile, loc=value, scale=err))

    return quantiles

# Generate quantiles for a given list of values and errors
def generate_list_quantiles(lst, err_lst):
    quantiles = []
    for i in range(len(lst)):
        quantiles.append(generate_quantiles(lst[i], err_lst[i]))

    return quantiles

def get_id_list():
    return sample_submission["id"].values

def extract_date_from_id(row_id):
    split = row_id.split('-')
    return '-'.join(split[:-1])

def extract_fips_from_id(row_id):
    return row_id.split('-')[-1]

# Get all dates used over the course of the term
all_dates = sample_submission["id"].values.copy()

for i in range(len(all_dates)):
    all_dates[i] = extract_date_from_id(all_dates[i])

# Remove duplicates in the list
all_dates = list(dict.fromkeys(all_dates))

# Given a start date and an end date in yyyy/mm/dd format,
# return a list of the dates within that range that includes
# both endpoints
def get_dates_in_range(startDate, endDate):
    dates = all_dates[all_dates.index(startDate):all_dates.index(endDate) + 1]
    return dates

# Starting from a given date, take an input number of steps
# and compute a list of dates containg the start date and
# "steps" dates into the future, for a total of steps
# dates.
def get_dates_from_start(startDate, steps):
    dates = all_dates[all_dates.index(startDate):all_dates.index(startDate) + steps]
    return dates

# Get the next date of a given date
def get_next_date(startDate):
    return get_dates_from_start(startDate, 2)[1]

# Get a list of all the deaths by date for a given county,
# starting from the date of the first case
def get_deaths_list(FIPS, endDate=convert_date_to_yyyy_mm_dd(today)):
    # Extract only the rows for this county in order by date
    rows = daily_deaths.loc[daily_deaths["fips"] == FIPS]
    deaths_list = rows["deaths"].values
    dates_list = rows["date"].values
    
    if endDate in dates_list:
        index = list(dates_list).index(endDate)
    else:
        return []
    
    return deaths_list[0:index+1]

Today: 5/9/20
One week ago: 5/2/20


In [26]:
lists = []

for row_id in get_id_list():
    date = extract_date_from_id(row_id)
    fips = int(extract_fips_from_id(row_id))
    
    if not fips in data:
        lst = [row_id] + ["%.2f" % 0.00] * 9
        lists.append(lst)
        continue
        
    if not date in data[fips]:
        lst = [row_id] + ["%.2f" % 0.00] * 9
        lists.append(lst)
        continue
    
    quantiles = data[fips][date]
    lst = [row_id]
    for q in quantiles:
        if str(q) == "nan":
            lst.append("%.2f" % 0.00)
        elif q < 0:
            lst.append("%.2f" % 0.00)
        else:
            lst.append("%.2f" % q)
            
    lists.append(lst)
    
df = pd.DataFrame(lists, columns=sample_submission.columns)

df.to_csv("prophet_checkpoint2_submission.csv", index=False, sep=',')